# Capstone Project Week 3 Assignment Part I

## 1. Transform Wikipedia page data into Dataframe

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [2]:
from IPython.display import display_html
import pandas as pd

unfiltered_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0] #read HTML from wikipedia
unfiltered_df.columns = ['PostalCode', 'Borough', 'Neighborhood'] #The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
unfiltered_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


As we see table above. There are NaN data on the Table. So we replace it to 'Not assigned'

In [3]:
unfiltered_df["Neighborhood"] = unfiltered_df["Neighborhood"].fillna('Not assigned') #remove NaN to 'Not Assigned'
unfiltered_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Drop rows which Borough equals Not assigned:

In [4]:
indexNames = unfiltered_df[ unfiltered_df['Borough'] == 'Not assigned' ].index
unfiltered_df.drop(indexNames , inplace=True)
unfiltered_df = unfiltered_df.reset_index(drop=True)
unfiltered_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


After that, group Neighborhood by PostalCode

In [5]:
unfiltered_df = unfiltered_df.groupby('PostalCode', as_index=False).agg(lambda x : ','.join(set(x)))
unfiltered_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Finally, filter the PostalCodes required by the assignment and order them accordingly:

In [6]:
filtered_df = unfiltered_df[unfiltered_df['PostalCode'].isin(['M5G', 'M2H','M4B','M1J','M4G','M4M','M1R','M9V','M9L','M5V','M1B','M5A']) ]
filtered_df = filtered_df.reset_index(drop=True)
filtered_df = filtered_df.loc[[8,3,4,1,5,6,2,11,10,9,0,7], :].reset_index(drop=True)
filtered_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,Parkview Hill / Woodbine Gardens
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,Wexford / Maryvale
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...


In [7]:
print('The dataframe shape is: ', filtered_df.shape)

The dataframe shape is:  (12, 3)


## 2. Add latitude and longitude from Geocoder

Use the Geocoder package or the csv file to create the dataframe.

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [8]:
!wget -q -O 'geospatialdata.csv' https://cocl.us/Geospatial_data

In [9]:
df_geospatial = pd.read_csv('geospatialdata.csv') #read csv from geospatialdata
df_geospatial.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df_geospatial.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
filtered_df = pd.merge(filtered_df, df_geospatial[['PostalCode', 'Latitude', 'Longitude']], on='PostalCode') #merge data
filtered_df[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']] #show data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,Wexford / Maryvale,43.750072,-79.295849
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...,43.628947,-79.394420


In [12]:
print('The dataframe shape is: ', filtered_df.shape)

The dataframe shape is:  (12, 5)
